<h2><center>California K-12 Public Schools</center></h2>
<h1><center>Meeting the Standards Projection</center></h1>


   <img src='images/MDR-education-data-hygiene.jpg' width="900">
   
   **Credit:**  [mdr education](https://mdreducation.com/2018/04/05/5-tips-spring-clean-education-data/)



In [1]:
# Load relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
import warnings

sns.set(style='ticks')

warnings.filterwarnings("ignore")  # Suppress all warnings

<h2><center>LANGUAGE ARTS & LITERATURE DATASET</center></h2>


# 1. DATA CLEANING

## Data definition
1. Column name
2. Data type
3. Description of column
4. Count or percent per unique values or code (includes NA)
5. Range of values

## Handling missing data
1. Identify how many NA are in the dataset
df.info()
.isnull()
value_counts()
2. Review the percentage of observatios missing per column
3. Drop, impute, or replace missing values

## Removing duplicates
- df.duplicated().sum()
- df.drop_duplicates()

In [2]:
# Load datafile
df_language = pd.read_csv('data/df_language')

# Check the dataset
df_language.head()

,School Name,School Code,Zip Code,County Name,County Code,Latitude,Longitude,Median Household Income,CAASPP Reported Enrollment,Enrollment K-12,...,Hispanic,Pacific Islander,White,Two/More Races,< High School,High School Grad,Some College,College Grad,Graduate School,Percentage Standard Met and Above
0,21st Century Learning Institute,129882,92223,Riverside,33.0,33.962281,-116.984589,64738.0,58,88.0,...,33,NaN,18,NaN,6,13,22,9,4,44.64
1,A. E. Arnold Elementary,6027767,90630,Orange,30.0,33.824900,-118.045700,84051.0,447,739.0,...,131,*,111,7,15,44,85,138,121,65.60
2,A. G. Cook Elementary,6028211,92844,Orange,30.0,33.777700,-117.953000,48345.0,192,366.0,...,43,NaN,10,6,*,13,14,33,6,81.18
3,A. G. Currie Middle,6085377,92780,Orange,30.0,33.743100,-117.824900,64089.0,585,611.0,...,532,*,15,*,223,168,82,39,15,30.38
4,A. J. Dorsa Elementary,6046114,95122,Santa Clara,43.0,37.369388,-121.833560,57470.0,184,371.0,...,166,NaN,*,*,82,55,24,15,*,29.83


In [3]:
# Column names
df_language.columns

Index(['School Name', 'School Code', 'Zip Code', 'County Name', 'County Code',
       'Latitude', 'Longitude', 'Median Household Income',
       'CAASPP Reported Enrollment', 'Enrollment K-12',
       'Total Revenue per Pupil', 'Total Expenditures per Pupil',
       'Free Meal Count K-12', 'Current Expense Per ADA', 'Male', 'Female',
       'Military', 'Non Military', 'Homeless', 'Non Homeless', 'Disadvantaged',
       'Not Disadvantaged', 'Black', 'Native American', 'Asian', 'Hispanic',
       'Pacific Islander', 'White', 'Two/More Races', '< High School',
       'High School Grad', 'Some College', 'College Grad', 'Graduate School',
       'Percentage Standard Met and Above'],
      dtype='object')

In [4]:
# Check data type
df_language.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10435 entries, 0 to 10434
Data columns (total 35 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   School Name                        10435 non-null  object 
 1   School Code                        10435 non-null  int64  
 2   Zip Code                           10435 non-null  int64  
 3   County Name                        10435 non-null  object 
 4   County Code                        10434 non-null  float64
 5   Latitude                           9310 non-null   float64
 6   Longitude                          9310 non-null   float64
 7   Median Household Income            10367 non-null  float64
 8   CAASPP Reported Enrollment         10434 non-null  object 
 9   Enrollment K-12                    9992 non-null   float64
 10  Total Revenue per Pupil            9310 non-null   object 
 11  Total Expenditures per Pupil       9310 non-null   obj

In [5]:
# Check missing data
df_language.isnull().sum().sort_values()

School Name                             0
School Code                             0
Zip Code                                0
County Name                             0
CAASPP Reported Enrollment              1
Non Military                            1
Percentage Standard Met and Above       1
County Code                             1
Non Homeless                            5
Male                                   51
Median Household Income                68
Disadvantaged                         149
Not Disadvantaged                     197
Hispanic                              214
Female                                253
Some College                          367
High School Grad                      392
Free Meal Count K-12                  443
Enrollment K-12                       443
Current Expense Per ADA               505
College Grad                          537
White                                 567
< High School                         948
Graduate School                   

## Handling Missing Data

1. Update data types:
    - Update values to int64, string or float as appropriate.
2. Create Dummy columns:
     - Add a column to flag missing data before imputation.
         - Existing values = 1
         - Missing values (NaN and *) = 0
3. Imputation:
    - Fill missing values with appropriate method.
    - Differentiate NaN from other types of missing data:
        - NaNs = -1
        - * = 0

In [6]:
# Turn following columns into numeric
cols = ['Total Revenue per Pupil', 'Total Expenditures per Pupil', 'Percentage Standard Met and Above']
df_language[cols] = df_language[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [7]:
# drop rows where there's no value for the dependent variables
df_language = df_language[df_language['Percentage Standard Met and Above'].notna()]

In [8]:
# Adding columns with dummy variables to flag missing values
cols = ['CAASPP Reported Enrollment', 'Enrollment K-12','Male', 'Female', 'Military', 
       'Homeless', 'Non Homeless', 'Disadvantaged', 'Not Disadvantaged', 'Non Military',
       'Black', 'Native American', 'Asian', 'Hispanic', 'Pacific Islander',
       'White', 'Two/More Races', '< High School', 'High School Grad',
       'Some College', 'College Grad', 'Graduate School']

def add_dummy(df, col):
    df[col + '-Dummy'] = df.apply(lambda row: 0 if pd.isnull(row[col]) or row[col] == '*' else 1, axis = 1)
    
for col in cols:
    add_dummy(df_language, col)

In [9]:
# Checking for added columns
df_language.columns

Index(['School Name', 'School Code', 'Zip Code', 'County Name', 'County Code',
       'Latitude', 'Longitude', 'Median Household Income',
       'CAASPP Reported Enrollment', 'Enrollment K-12',
       'Total Revenue per Pupil', 'Total Expenditures per Pupil',
       'Free Meal Count K-12', 'Current Expense Per ADA', 'Male', 'Female',
       'Military', 'Non Military', 'Homeless', 'Non Homeless', 'Disadvantaged',
       'Not Disadvantaged', 'Black', 'Native American', 'Asian', 'Hispanic',
       'Pacific Islander', 'White', 'Two/More Races', '< High School',
       'High School Grad', 'Some College', 'College Grad', 'Graduate School',
       'Percentage Standard Met and Above', 'CAASPP Reported Enrollment-Dummy',
       'Enrollment K-12-Dummy', 'Male-Dummy', 'Female-Dummy', 'Military-Dummy',
       'Homeless-Dummy', 'Non Homeless-Dummy', 'Disadvantaged-Dummy',
       'Not Disadvantaged-Dummy', 'Non Military-Dummy', 'Black-Dummy',
       'Native American-Dummy', 'Asian-Dummy', 'Hispanic-

In [10]:
# Rearrange columns order
df_language = df_language[['School Name', 'School Code', 'Zip Code', 'County Name', 'County Code',
       'Latitude', 'Longitude', 'Median Household Income',
       'CAASPP Reported Enrollment', 'CAASPP Reported Enrollment-Dummy', 'Enrollment K-12',
       'Enrollment K-12-Dummy', 'Total Revenue per Pupil', 'Total Expenditures per Pupil',
       'Free Meal Count K-12', 'Current Expense Per ADA', 
       'Male', 'Male-Dummy', 'Female', 'Female-Dummy', 'Military', 'Military-Dummy',
       'Non Military', 'Non Military-Dummy','Homeless', 'Homeless-Dummy', 
       'Non Homeless', 'Non Homeless-Dummy', 'Disadvantaged', 'Disadvantaged-Dummy',
       'Not Disadvantaged', 'Not Disadvantaged-Dummy', 'Black', 'Black-Dummy',
       'Native American', 'Native American-Dummy', 'Asian', 'Asian-Dummy', 
       'Hispanic', 'Hispanic-Dummy', 'Pacific Islander', 'Pacific Islander-Dummy',
       'White', 'White-Dummy', 'Two/More Races', 'Two/More Races-Dummy', 
       '< High School', '< High School-Dummy', 'High School Grad', 'High School Grad-Dummy',
       'Some College', 'Some College-Dummy', 'College Grad', 'College Grad-Dummy', 
       'Graduate School', 'Graduate School-Dummy','Percentage Standard Met and Above']]
df_language.head()

,School Name,School Code,Zip Code,County Name,County Code,Latitude,Longitude,Median Household Income,CAASPP Reported Enrollment,CAASPP Reported Enrollment-Dummy,...,< High School-Dummy,High School Grad,High School Grad-Dummy,Some College,Some College-Dummy,College Grad,College Grad-Dummy,Graduate School,Graduate School-Dummy,Percentage Standard Met and Above
0,21st Century Learning Institute,129882,92223,Riverside,33.0,33.962281,-116.984589,64738.0,58,1,...,1,13,1,22,1,9,1,4,1,44.64
1,A. E. Arnold Elementary,6027767,90630,Orange,30.0,33.824900,-118.045700,84051.0,447,1,...,1,44,1,85,1,138,1,121,1,65.60
2,A. G. Cook Elementary,6028211,92844,Orange,30.0,33.777700,-117.953000,48345.0,192,1,...,0,13,1,14,1,33,1,6,1,81.18
3,A. G. Currie Middle,6085377,92780,Orange,30.0,33.743100,-117.824900,64089.0,585,1,...,1,168,1,82,1,39,1,15,1,30.38
4,A. J. Dorsa Elementary,6046114,95122,Santa Clara,43.0,37.369388,-121.833560,57470.0,184,1,...,1,55,1,24,1,15,1,*,0,29.83


In [11]:
# Verifying if code worked 
df_language[['Native American', 'Native American-Dummy', 'Asian', 'Asian-Dummy', 
       'Hispanic', 'Hispanic-Dummy', 'Pacific Islander', 'Pacific Islander-Dummy']]

,Native American,Native American-Dummy,Asian,Asian-Dummy,Hispanic,Hispanic-Dummy,Pacific Islander,Pacific Islander-Dummy
0,NaN,0,*,0,33,1,NaN,0
1,NaN,0,150,1,131,1,*,0
2,NaN,0,133,1,43,1,NaN,0
3,*,0,18,1,532,1,*,0
4,NaN,0,8,1,166,1,NaN,0
...,...,...,...,...,...,...,...,...
10430,NaN,0,*,0,14,1,NaN,0
10431,NaN,0,NaN,0,24,1,NaN,0
10432,14,1,15,1,36,1,8,1
10433,*,0,*,0,112,1,*,0


In [12]:
# Fill demographic columns NaN with 0 and * with -1
df_language.update(df_language[['CAASPP Reported Enrollment', 'Latitude',
       'Enrollment K-12','Male', 'Female', 'Military', 'Non Military', 'Longitude',
       'Homeless', 'Non Homeless', 'Disadvantaged', 'Not Disadvantaged',
       'Black', 'Native American', 'Asian', 'Hispanic', 'Pacific Islander',
       'White', 'Two/More Races', '< High School', 'High School Grad',
       'Some College', 'College Grad', 'Graduate School']].replace('*', 0).fillna(-1))

In [13]:
# Turn following columns into numeric
cols = ['CAASPP Reported Enrollment', 'County Code', 'Latitude',
       'Enrollment K-12','Male', 'Female', 'Military', 'Non Military', 'Longitude',
       'Homeless', 'Non Homeless', 'Disadvantaged', 'Not Disadvantaged',
       'Black', 'Native American', 'Asian', 'Hispanic', 'Pacific Islander',
       'White', 'Two/More Races', '< High School', 'High School Grad',
       'Some College', 'College Grad', 'Graduate School']

df_language[cols] = df_language[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [14]:
# # finding 90th for each of following variables ----> Used this to create csv for graphing only
# revenue_per = language['Total Revenue per Pupil'].quantile(0.95)
# expenditure_per = language['Total Expenditures per Pupil'].quantile(0.95)
# ada_expense_per = language['Current Expense Per ADA'].quantile(0.95).round(2)

# # Replace values above x with 95th percentile
# language['Total Revenue per Pupil'] = language['Total Revenue per Pupil'].map(
#     lambda x: revenue_per if x > 20000 else x).round(2)
# language['Total Expenditures per Pupil'] = language['Total Expenditures per Pupil'].map(
#     lambda x: expenditure_per if x > 20000 else x).round(2)
# language['Current Expense Per ADA'] = language['Current Expense Per ADA'].map(
#     lambda x: ada_expense_per if x > 20000 else x).round(2)


In [15]:
# Impute missing data with median for the following columns
df_language['Median Household Income'].fillna(df_language['Median Household Income'].median(), inplace=True)
df_language['Free Meal Count K-12'].fillna(df_language['Free Meal Count K-12'].median(), inplace=True)
df_language['Current Expense Per ADA'].fillna(df_language['Current Expense Per ADA'].median(), inplace=True)
df_language['Total Revenue per Pupil'].fillna(df_language['Total Revenue per Pupil'].median(), inplace=True)
df_language['Total Expenditures per Pupil'].fillna(df_language['Total Expenditures per Pupil'].median(), inplace=True)

# percentage of missing data per column
percent_missing = (df_language.isnull().sum() * 100 / len(df_language)).round(2)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})

# sorting values in ascending format
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df

,percent_missing
School Name,0.0
Not Disadvantaged,0.0
Not Disadvantaged-Dummy,0.0
Black,0.0
Black-Dummy,0.0
Native American,0.0
Native American-Dummy,0.0
Asian,0.0
Asian-Dummy,0.0
Hispanic,0.0


In [16]:
# Check for duplicate
df_language.duplicated().sum()

0

# 2. DATA TRANSFORMATION

1. Transform columns from number of students to percentage
2. Update missing value back to -1
3. Create csv file for pre-processing and modeling

In [17]:
# make copy of df_language to transform selected column into percentage values
language = df_language.copy()

#language.head()

In [18]:
# Function to transform values in percentage
def percentage(df, demographic):
    df[demographic] = ((df[demographic] / df['CAASPP Reported Enrollment']).round(4))*100

# List with column names to modify into percentage values
demographics = ['Male', 'Female', 'Military', 'Non Military',
       'Homeless', 'Non Homeless', 'Disadvantaged', 'Not Disadvantaged',
       'Black', 'Native American', 'Asian', 'Hispanic', 'Pacific Islander',
       'White', 'Two/More Races', '< High School', 'High School Grad',
       'Some College', 'College Grad', 'Graduate School']

# Loop through list apply percentage()
for i in demographics:
    percentage(language, i)


In [19]:
# Turn negative values back to -1 to represent missing data
cols = ['Enrollment K-12','Male', 'Female', 'Military', 'Non Military',
       'Homeless', 'Non Homeless', 'Disadvantaged', 'Not Disadvantaged',
       'Black', 'Native American', 'Asian', 'Hispanic', 'Pacific Islander',
       'White', 'Two/More Races', '< High School', 'High School Grad',
       'Some College', 'College Grad', 'Graduate School']

def map_values(df, col):
    df[col] = df[col].map(lambda x: -1 if x < 0 else x).round(2)
    
for col in cols:
    map_values(language, col)


# See final version
language.head(10)

# # create csv file with percentage values
# language.to_csv("df_language_with_dummies", encoding='utf-8', index=False)

,School Name,School Code,Zip Code,County Name,County Code,Latitude,Longitude,Median Household Income,CAASPP Reported Enrollment,CAASPP Reported Enrollment-Dummy,...,< High School-Dummy,High School Grad,High School Grad-Dummy,Some College,Some College-Dummy,College Grad,College Grad-Dummy,Graduate School,Graduate School-Dummy,Percentage Standard Met and Above
0,21st Century Learning Institute,129882,92223,Riverside,33.0,33.962281,-116.984589,64738.0,58.0,1,...,1,22.41,1,37.93,1,15.52,1,6.90,1,44.64
1,A. E. Arnold Elementary,6027767,90630,Orange,30.0,33.824900,-118.045700,84051.0,447.0,1,...,1,9.84,1,19.02,1,30.87,1,27.07,1,65.60
2,A. G. Cook Elementary,6028211,92844,Orange,30.0,33.777700,-117.953000,48345.0,192.0,1,...,0,6.77,1,7.29,1,17.19,1,3.12,1,81.18
3,A. G. Currie Middle,6085377,92780,Orange,30.0,33.743100,-117.824900,64089.0,585.0,1,...,1,28.72,1,14.02,1,6.67,1,2.56,1,30.38
4,A. J. Dorsa Elementary,6046114,95122,Santa Clara,43.0,37.369388,-121.833560,57470.0,184.0,1,...,1,29.89,1,13.04,1,8.15,1,0.00,0,29.83
5,A. L. Conner Elementary,105692,93646,Fresno,10.0,36.596730,-119.453485,27914.0,172.0,1,...,1,31.40,1,19.19,1,9.88,1,0.00,0,45.88
6,A. M. Thomas Middle,6102792,93249,Kern,15.0,35.615270,-119.702911,33011.0,162.0,1,...,1,34.57,1,3.09,1,0.00,0,-1.00,0,26.09
7,A. M. Winn Waldorf-Inspired,6033765,95827,Sacramento,34.0,38.511420,-121.438441,50528.0,214.0,1,...,1,30.37,1,34.11,1,16.82,1,5.61,1,18.48
8,ABC Secondary (Alternative),1995596,90703,Los Angeles,19.0,33.879715,-118.071463,90613.0,37.0,1,...,1,27.03,1,0.00,0,0.00,0,0.00,0,22.22
10,ACCESS County Community,3030764,92628,Orange,30.0,-1.000000,-1.000000,56671.0,602.0,1,...,1,14.12,1,16.78,1,9.30,1,2.82,1,11.56


-------

<center><h2>MATHEMATICS DATASET</center></h2>

# 1. DATA CLEANING

## Data definition
1. Column name
2. Data type
3. Description of column
4. Count or percent per unique values or code (includes NA)
5. Range of values

## Handling missing data
1. Identify how many NA are in the dataset
df.info()
.isnull()
value_counts()
2. Review the percentage of observatios missing per column
3. Drop, impute, or replace missing values

## Removing duplicates
- df.duplicated().sum()
- df.drop_duplicates()

In [20]:
# load datafile
df_math = pd.read_csv('data/df_math')

# Check df
df_math.head()

,School Name,School Code,Zip Code,County Name,County Code,Latitude,Longitude,Median Household Income,CAASPP Reported Enrollment,Enrollment K-12,...,Hispanic,Pacific Islander,White,Two/More Races,< High School,High School Grad,Some College,College Grad,Graduate School,Percentage Standard Met and Above
0,21st Century Learning Institute,129882,92223,Riverside,33.0,33.962281,-116.984589,64738.0,58,88.0,...,33,NaN,18,NaN,6,13,22,9,4,10.71
1,A. E. Arnold Elementary,6027767,90630,Orange,30.0,33.824900,-118.045700,84051.0,447,739.0,...,131,*,111,7,15,44,85,138,121,63.41
2,A. G. Cook Elementary,6028211,92844,Orange,30.0,33.777700,-117.953000,48345.0,192,366.0,...,43,NaN,10,6,*,13,14,33,6,71.81
3,A. G. Currie Middle,6085377,92780,Orange,30.0,33.743100,-117.824900,64089.0,585,611.0,...,532,*,15,*,223,168,82,39,15,18.35
4,A. J. Dorsa Elementary,6046114,95122,Santa Clara,43.0,37.369388,-121.833560,57470.0,184,371.0,...,166,NaN,*,*,82,55,24,15,*,25.41


In [21]:
# Check data types
df_math.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10435 entries, 0 to 10434
Data columns (total 35 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   School Name                        10435 non-null  object 
 1   School Code                        10435 non-null  int64  
 2   Zip Code                           10435 non-null  int64  
 3   County Name                        10435 non-null  object 
 4   County Code                        10433 non-null  float64
 5   Latitude                           9310 non-null   float64
 6   Longitude                          9310 non-null   float64
 7   Median Household Income            10367 non-null  float64
 8   CAASPP Reported Enrollment         10433 non-null  object 
 9   Enrollment K-12                    9992 non-null   float64
 10  Total Revenue per Pupil            9310 non-null   object 
 11  Total Expenditures per Pupil       9310 non-null   obj

In [22]:
# Missing values
df_math.isnull().sum().sort_values()

School Name                             0
School Code                             0
Zip Code                                0
County Name                             0
CAASPP Reported Enrollment              2
Non Military                            2
Percentage Standard Met and Above       2
County Code                             2
Non Homeless                            7
Male                                   53
Median Household Income                68
Disadvantaged                         149
Not Disadvantaged                     196
Hispanic                              216
Female                                255
Some College                          368
High School Grad                      390
Free Meal Count K-12                  443
Enrollment K-12                       443
Current Expense Per ADA               505
College Grad                          536
White                                 569
< High School                         948
Graduate School                   

## Handling Missing Data

1. Update data types:
    - Update values to int64, string or float as appropriate.
2. Create Dummy columns:
     - Add a column to flag missing data before imputation.
         - Existing values = 1
         - Missing values (NaN and *) = 0
3. Imputation:
    - Fill missing values with appropriate method.
    - Differentiate NaN from other types of missing data:
        - NaNs = 0
        - * = -1

In [23]:
# Turn the following columns to numeric
cols = ['Total Revenue per Pupil', 'Total Expenditures per Pupil', 'Percentage Standard Met and Above']
df_math[cols] = df_math[cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Drop rows where dependent variable missing
df_math = df_math[df_math['Percentage Standard Met and Above'].notna()]

In [24]:
# Adding columns with dummy variables to flag missing values
cols = ['CAASPP Reported Enrollment', 'Enrollment K-12','Male', 'Female', 'Military', 
       'Homeless', 'Non Homeless', 'Disadvantaged', 'Not Disadvantaged', 'Non Military',
       'Black', 'Native American', 'Asian', 'Hispanic', 'Pacific Islander',
       'White', 'Two/More Races', '< High School', 'High School Grad',
       'Some College', 'College Grad', 'Graduate School']

for col in cols:
    add_dummy(df_math, col)

# Check df    
df_math

,School Name,School Code,Zip Code,County Name,County Code,Latitude,Longitude,Median Household Income,CAASPP Reported Enrollment,Enrollment K-12,...,Asian-Dummy,Hispanic-Dummy,Pacific Islander-Dummy,White-Dummy,Two/More Races-Dummy,< High School-Dummy,High School Grad-Dummy,Some College-Dummy,College Grad-Dummy,Graduate School-Dummy
0,21st Century Learning Institute,129882,92223,Riverside,33.0,33.962281,-116.984589,64738.0,58,88.0,...,0,1,0,1,0,1,1,1,1,1
1,A. E. Arnold Elementary,6027767,90630,Orange,30.0,33.824900,-118.045700,84051.0,447,739.0,...,1,1,0,1,1,1,1,1,1,1
2,A. G. Cook Elementary,6028211,92844,Orange,30.0,33.777700,-117.953000,48345.0,192,366.0,...,1,1,0,1,1,0,1,1,1,1
3,A. G. Currie Middle,6085377,92780,Orange,30.0,33.743100,-117.824900,64089.0,585,611.0,...,1,1,0,1,0,1,1,1,1,1
4,A. J. Dorsa Elementary,6046114,95122,Santa Clara,43.0,37.369388,-121.833560,57470.0,184,371.0,...,1,1,0,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10430,iLEAD Online,136531,93510,Los Angeles,19.0,34.472708,-118.196768,89403.0,52,73.0,...,0,1,0,1,0,0,0,1,1,1
10431,iLead Agua Dulce,138297,91390,Los Angeles,19.0,34.472708,-118.196768,105659.0,64,119.0,...,0,1,0,1,1,0,0,1,1,1
10432,iQ Academy California-Los Angeles,120600,93065,Los Angeles,19.0,33.985314,-117.888584,94173.0,405,702.0,...,1,1,1,1,1,0,1,1,1,1
10433,one.Charter,120717,95206,San Joaquin,39.0,NaN,NaN,42404.0,184,509.0,...,0,1,0,1,1,1,1,1,1,1


In [25]:
df_math.columns

Index(['School Name', 'School Code', 'Zip Code', 'County Name', 'County Code',
       'Latitude', 'Longitude', 'Median Household Income',
       'CAASPP Reported Enrollment', 'Enrollment K-12',
       'Total Revenue per Pupil', 'Total Expenditures per Pupil',
       'Free Meal Count K-12', 'Current Expense Per ADA', 'Male', 'Female',
       'Military', 'Non Military', 'Homeless', 'Non Homeless', 'Disadvantaged',
       'Not Disadvantaged', 'Black', 'Native American', 'Asian', 'Hispanic',
       'Pacific Islander', 'White', 'Two/More Races', '< High School',
       'High School Grad', 'Some College', 'College Grad', 'Graduate School',
       'Percentage Standard Met and Above', 'CAASPP Reported Enrollment-Dummy',
       'Enrollment K-12-Dummy', 'Male-Dummy', 'Female-Dummy', 'Military-Dummy',
       'Homeless-Dummy', 'Non Homeless-Dummy', 'Disadvantaged-Dummy',
       'Not Disadvantaged-Dummy', 'Non Military-Dummy', 'Black-Dummy',
       'Native American-Dummy', 'Asian-Dummy', 'Hispanic-

In [26]:
# Rearrange columns order
df_math = df_math[['School Name', 'School Code', 'Zip Code', 'County Name', 'County Code',
       'Latitude', 'Longitude', 'Median Household Income',
       'CAASPP Reported Enrollment', 'CAASPP Reported Enrollment-Dummy', 'Enrollment K-12',
       'Enrollment K-12-Dummy', 'Total Revenue per Pupil', 'Total Expenditures per Pupil',
       'Free Meal Count K-12', 'Current Expense Per ADA', 
       'Male', 'Male-Dummy', 'Female', 'Female-Dummy', 'Military', 'Military-Dummy',
       'Non Military', 'Non Military-Dummy','Homeless', 'Homeless-Dummy', 
       'Non Homeless', 'Non Homeless-Dummy', 'Disadvantaged', 'Disadvantaged-Dummy',
       'Not Disadvantaged', 'Not Disadvantaged-Dummy', 'Black', 'Black-Dummy',
       'Native American', 'Native American-Dummy', 'Asian', 'Asian-Dummy', 
       'Hispanic', 'Hispanic-Dummy', 'Pacific Islander', 'Pacific Islander-Dummy',
       'White', 'White-Dummy', 'Two/More Races', 'Two/More Races-Dummy', 
       '< High School', '< High School-Dummy', 'High School Grad', 'High School Grad-Dummy',
       'Some College', 'Some College-Dummy', 'College Grad', 'College Grad-Dummy', 
       'Graduate School', 'Graduate School-Dummy','Percentage Standard Met and Above']]
df_math.head()

,School Name,School Code,Zip Code,County Name,County Code,Latitude,Longitude,Median Household Income,CAASPP Reported Enrollment,CAASPP Reported Enrollment-Dummy,...,< High School-Dummy,High School Grad,High School Grad-Dummy,Some College,Some College-Dummy,College Grad,College Grad-Dummy,Graduate School,Graduate School-Dummy,Percentage Standard Met and Above
0,21st Century Learning Institute,129882,92223,Riverside,33.0,33.962281,-116.984589,64738.0,58,1,...,1,13,1,22,1,9,1,4,1,10.71
1,A. E. Arnold Elementary,6027767,90630,Orange,30.0,33.824900,-118.045700,84051.0,447,1,...,1,44,1,85,1,138,1,121,1,63.41
2,A. G. Cook Elementary,6028211,92844,Orange,30.0,33.777700,-117.953000,48345.0,192,1,...,0,13,1,14,1,33,1,6,1,71.81
3,A. G. Currie Middle,6085377,92780,Orange,30.0,33.743100,-117.824900,64089.0,585,1,...,1,168,1,82,1,39,1,15,1,18.35
4,A. J. Dorsa Elementary,6046114,95122,Santa Clara,43.0,37.369388,-121.833560,57470.0,184,1,...,1,55,1,24,1,15,1,*,0,25.41


In [27]:
# Replace missing values * with -1 anmd NaN woith 0
df_math.update(df_math[['CAASPP Reported Enrollment', 'Latitude',
       'Enrollment K-12','Male', 'Female', 'Military', 'Non Military', 'Longitude',
       'Homeless', 'Non Homeless', 'Disadvantaged', 'Not Disadvantaged',
       'Black', 'Native American', 'Asian', 'Hispanic', 'Pacific Islander',
       'White', 'Two/More Races', '< High School', 'High School Grad',
       'Some College', 'College Grad', 'Graduate School']].replace('*', -1).fillna(0))

In [28]:
# Turn following columns into numeric
cols = ['CAASPP Reported Enrollment', 'County Code', 'Latitude',
       'Enrollment K-12','Male', 'Female', 'Military', 'Non Military', 'Longitude',
       'Homeless', 'Non Homeless', 'Disadvantaged', 'Not Disadvantaged',
       'Black', 'Native American', 'Asian', 'Hispanic', 'Pacific Islander',
       'White', 'Two/More Races', '< High School', 'High School Grad',
       'Some College', 'College Grad', 'Graduate School']

df_math[cols] = df_math[cols].apply(pd.to_numeric, errors='coerce', axis=1)

df_math.head()

,School Name,School Code,Zip Code,County Name,County Code,Latitude,Longitude,Median Household Income,CAASPP Reported Enrollment,CAASPP Reported Enrollment-Dummy,...,< High School-Dummy,High School Grad,High School Grad-Dummy,Some College,Some College-Dummy,College Grad,College Grad-Dummy,Graduate School,Graduate School-Dummy,Percentage Standard Met and Above
0,21st Century Learning Institute,129882,92223,Riverside,33.0,33.962281,-116.984589,64738.0,58.0,1,...,1,13.0,1,22.0,1,9.0,1,4.0,1,10.71
1,A. E. Arnold Elementary,6027767,90630,Orange,30.0,33.824900,-118.045700,84051.0,447.0,1,...,1,44.0,1,85.0,1,138.0,1,121.0,1,63.41
2,A. G. Cook Elementary,6028211,92844,Orange,30.0,33.777700,-117.953000,48345.0,192.0,1,...,0,13.0,1,14.0,1,33.0,1,6.0,1,71.81
3,A. G. Currie Middle,6085377,92780,Orange,30.0,33.743100,-117.824900,64089.0,585.0,1,...,1,168.0,1,82.0,1,39.0,1,15.0,1,18.35
4,A. J. Dorsa Elementary,6046114,95122,Santa Clara,43.0,37.369388,-121.833560,57470.0,184.0,1,...,1,55.0,1,24.0,1,15.0,1,-1.0,0,25.41


In [29]:
# # finding 90th for each variable ---> Needs to be updated to df_math
# revenue_per = language['Total Revenue per Pupil'].quantile(0.95)
# expenditure_per = language['Total Expenditures per Pupil'].quantile(0.95)
# ada_expense_per = language['Current Expense Per ADA'].quantile(0.95).round(2)

# # Replace values above x with 95th percentile
# language['Total Revenue per Pupil'] = language['Total Revenue per Pupil'].map(
#     lambda x: revenue_per if x > 20000 else x).round(2)
# language['Total Expenditures per Pupil'] = language['Total Expenditures per Pupil'].map(
#     lambda x: expenditure_per if x > 20000 else x).round(2)
# language['Current Expense Per ADA'] = language['Current Expense Per ADA'].map(
#     lambda x: ada_expense_per if x > 20000 else x).round(2)


In [30]:
# Impute missing data with median for the following columns
df_math['Median Household Income'].fillna(df_math['Median Household Income'].median(), inplace=True)
df_math['Free Meal Count K-12'].fillna(df_math['Free Meal Count K-12'].median(), inplace=True)
df_math['Current Expense Per ADA'].fillna(df_math['Current Expense Per ADA'].median(), inplace=True)
df_math['Total Revenue per Pupil'].fillna(df_math['Total Revenue per Pupil'].median(), inplace=True)
df_math['Total Expenditures per Pupil'].fillna(df_math['Total Expenditures per Pupil'].median(), inplace=True)

# percentage of missing data per column
percent_missing = (df_math.isnull().sum() * 100 / len(df_math)).round(2)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})

# sorting values in ascending format
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df

,percent_missing
School Name,0.0
Not Disadvantaged,0.0
Not Disadvantaged-Dummy,0.0
Black,0.0
Black-Dummy,0.0
Native American,0.0
Native American-Dummy,0.0
Asian,0.0
Asian-Dummy,0.0
Hispanic,0.0


In [31]:
# Check for duplicate
df_math.duplicated().sum()

0

# 2. DATA TRANSFORMATION

1. Transform columns from number of students to percentage
2. Update missing value back to -1
3. Create csv file for pre-processing and modeling

In [32]:
# make copy of df to transform selected column into percentage values
math = df_math.copy()

#math.head()

In [33]:
# Loop through list apply percentage()
for i in demographics:
    percentage(math, i)

In [34]:
# Turn negative values back to -1 to represent missing data
cols = ['Enrollment K-12','Male', 'Female', 'Military', 'Non Military',
       'Homeless', 'Non Homeless', 'Disadvantaged', 'Not Disadvantaged',
       'Black', 'Native American', 'Asian', 'Hispanic', 'Pacific Islander',
       'White', 'Two/More Races', '< High School', 'High School Grad',
       'Some College', 'College Grad', 'Graduate School']

for col in cols:
    map_values(math, col)

# See final version
math.head(10)

# # create csv file with percentage values
# math.to_csv("df_math_with_dummies", encoding='utf-8', index=False)

,School Name,School Code,Zip Code,County Name,County Code,Latitude,Longitude,Median Household Income,CAASPP Reported Enrollment,CAASPP Reported Enrollment-Dummy,...,< High School-Dummy,High School Grad,High School Grad-Dummy,Some College,Some College-Dummy,College Grad,College Grad-Dummy,Graduate School,Graduate School-Dummy,Percentage Standard Met and Above
0,21st Century Learning Institute,129882,92223,Riverside,33.0,33.962281,-116.984589,64738.0,58.0,1,...,1,22.41,1,37.93,1,15.52,1,6.90,1,10.71
1,A. E. Arnold Elementary,6027767,90630,Orange,30.0,33.824900,-118.045700,84051.0,447.0,1,...,1,9.84,1,19.02,1,30.87,1,27.07,1,63.41
2,A. G. Cook Elementary,6028211,92844,Orange,30.0,33.777700,-117.953000,48345.0,192.0,1,...,0,6.77,1,7.29,1,17.19,1,3.12,1,71.81
3,A. G. Currie Middle,6085377,92780,Orange,30.0,33.743100,-117.824900,64089.0,585.0,1,...,1,28.72,1,14.02,1,6.67,1,2.56,1,18.35
4,A. J. Dorsa Elementary,6046114,95122,Santa Clara,43.0,37.369388,-121.833560,57470.0,184.0,1,...,1,29.89,1,13.04,1,8.15,1,-1.00,0,25.41
5,A. L. Conner Elementary,105692,93646,Fresno,10.0,36.596730,-119.453485,27914.0,172.0,1,...,1,31.40,1,19.19,1,9.88,1,-1.00,0,44.97
6,A. M. Thomas Middle,6102792,93249,Kern,15.0,35.615270,-119.702911,33011.0,162.0,1,...,1,34.57,1,3.09,1,-1.00,0,0.00,0,9.26
7,A. M. Winn Waldorf-Inspired,6033765,95827,Sacramento,34.0,38.511420,-121.438441,50528.0,214.0,1,...,1,30.37,1,34.11,1,16.82,1,5.61,1,10.95
8,ABC Secondary (Alternative),1995596,90703,Los Angeles,19.0,33.879715,-118.071463,90613.0,38.0,1,...,1,26.32,1,18.42,1,-1.00,0,-1.00,0,5.26
10,ACCESS County Community,3030764,92628,Orange,30.0,0.000000,0.000000,56671.0,602.0,1,...,1,14.12,1,16.78,1,9.30,1,2.82,1,0.87


------